In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Tue Jan 31 04:28:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
os.chdir('/content/drive/MyDrive/Study/Data Science/AI 자연어 처리 전문가 양성 과정 6기/Project 3')

In [4]:
!pip -q install konlpy transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 117.6 MB/s eta 0:00:00


In [25]:
from easydict import EasyDict as edict
from konlpy.tag import Okt
from nltk.translate.bleu_score import sentence_bleu
from transformers import (
    AutoTokenizer,
    BartForConditionalGeneration,
    T5ForConditionalGeneration
)
import numpy as np
import pandas as pd

In [32]:
source_lang = "spoken"
target_lang = "written"
model_checkpoint = f"yangdk/t5-base-korean-paraphrase-finetuned-spoken-to-written"

In [33]:
#tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')
tokenizer = AutoTokenizer.from_pretrained('lcw99/t5-base-korean-paraphrase')
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [34]:
df_validation = pd.read_csv('input/df_spoken-written_validation.csv', header=None)
df_validation.columns = ['id', 'spoken', 'written']

In [35]:
okt = Okt()
generated = []
blue_score = []
for idx, row in df_validation.iterrows():
    input_text = row[source_lang]
    tokenized = tokenizer(input_text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    tokenized = {k: v.cuda() for k, v in tokenized.items()}
    out = model.generate(**tokenized, max_length=128)[0, 1:-1].cpu()
    out = tokenizer.decode(out, skip_special_tokens=True)
    generated.append(out)

    candidate = okt.morphs(out)
    references = [row[target_lang]]
    references = [okt.morphs(s) for s in references]
    blue_score.append(sentence_bleu(references, candidate))

df_validation['generated'] = generated
df_validation['blue_score'] = blue_score

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [36]:
df_validation.loc[df_validation.blue_score > 0.5]

,id,spoken,written,generated,blue_score
3,4764,"교수님, 제 성적이 이상한 것 같아요.","교수님, 저는 성적이 이상하다고 생각합니다.","교수님, 제 성적이 이상하다고 생각합니다.",0.595417
4,9106,제가 동물을 좋아해서 그 개를 무척 좋아해요.,나는 동물을 좋아해서 개를 매우 좋아합니다.,나는 동물을 좋아하기 때문에 개를 매우 좋아합니다.,0.531073
10,1149,노력을 판단하기 위한 최선의 방법이에요.,노력을 판단하는 가장 좋은 방법입니다.,노력을 판단하는 가장 좋은 방법입니다.,1.000000
13,10139,내 마음속에도 빛이 있다면 좋겠어.,내 마음에 빛이 있었으면 좋겠다.,내 마음에 빛이 있다면 좋겠다.,0.594604
18,13740,우리의 최고 스포츠 선수들은 과거보다 훨씬 더 많은 돈을 받아요.,우리의 최고의 스포츠 선수들은 과거보다 훨씬 더 많은 돈을받습니다.,우리 최고의 스포츠 선수들은 과거보다 훨씬 더 많은 돈을 받습니다.,0.888442
...,...,...,...,...,...
2748,3590,나는 어렸을 때 가수가 되고 싶었어.,어렸을 때 가수가되고 싶었습니다.,어렸을 때 가수가 되고 싶었다.,0.643459
2751,14386,이 대답은 우리가 요구하는 것을 충족시키지 못하네요.,이 답변은 우리가 요구하는 것을 충족시키지 않습니다.,이 대답은 우리가 원하는 것을 충족시키지 않습니다.,0.501651
2752,9404,당신을 좋아하기 때문에 그렇게 말할 수밖에 없었어.,나는 당신을 좋아하기 때문에 그렇게 말할 수밖에 없었습니다.,나는 당신을 좋아하기 때문에 그렇게 말할 수 없었습니다.,0.814949
2753,13753,우리 회사는 네가 요청한 모든 것들을 잘 알고 있으니 걱정 말아.,우리 회사는 당신이 요청한 모든 것을 알고 있기 때문에 걱정하지 마십시오.,우리 회사는 당신이 요청한 모든 것을 잘 알고 있으므로 걱정하지 마십시오.,0.652646


In [37]:
df_validation.to_csv(f'output/{model_checkpoint.split("/")[-1]}_generated.csv', index=False)

In [38]:
np.mean(df_validation['blue_score'])

0.397609256295421